In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# # EXAMPLE CODE

# # IMPORTS:
# from wav_conversion import convert_mp3_to_wav
# from textgrid_creation import create_textgrids
# from custom_mfa_align import align
# from formant_extraction import extract

# # ARGUMENTS:
mp3_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/mp3-input"
in_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/montreal-input"
out_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class/montreal-output"
home_folder = "/home/henrynomeland/Documents/senior-thesis/speech-class" 
label = "Please call Stella. Ask her to bring these things with her from the store: \
Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for \
her brother Bob. We also need a small plastic snake and a big toy frog for the kids. She \
can scoop these things into three red bags, and we will go meet her Wednesday at the train station."
acoustic_model = "english_us_arpa"
dictionary_model = "english_us_arpa"
filename="testing.csv"

# # this converts mp3 files to wav files
# # convert_mp3_to_wav(mp3_folder, in_folder, out_folder)
# # this creates the textgrids
# create_textgrids(label, in_folder)
# # this aligns the textgrids
# align(in_folder, out_folder, acoustic_model, dictionary_model)
# # this calculates formant measurements
# extract(out_folder, home_folder, filename)

In [16]:
from speech_modeling import h_input, h_model
import pandas as pd
df = pd.read_csv('gmu-formants.csv', low_memory=False)
input_data = h_input(df)
input_data.process('region')
input_data.output_input_df()
input_data.select_features(selected_features=['F1', 'duration'])
input_data.normalize(method='z')
input_data.select_places(places=['uk', 'texas', 'california', 'newyork', 'minnesota'])

model_data = input_data.input_df.copy()
model_features = list(input_data.input_df.columns[4:])

m = h_model(data=model_data, features=model_features, y_feature="location", y_main="uk")
m.fit(model_type="rforest")

Fitting model of type rforest
Fitting LOOCV split 0
Fitting LOOCV split 10
Fitting LOOCV split 20
Fitting LOOCV split 30
Fitting LOOCV split 40
Fitting LOOCV split 50

Accuracy: 0.904

Variable Importance Measurements:
      Feature  Importance
63     du-bob    0.001331
80    du-need    0.001257
1     F1-also    0.001257
11    F1-call    0.001183
44   F1-these    0.001109
30     F1-red    0.001036
97   du-store    0.001036
60    du-bags    0.000740
48      F1-to    0.000666
29  F1-please    0.000666


In [17]:
m = h_model(data=model_data, features=model_features, y_feature="location", y_main="uk")
m.fit(model_type="ridge_classification")

Fitting model of type ridge_classification
Fitting LOOCV split 0
Fitting LOOCV split 10
Fitting LOOCV split 20
Fitting LOOCV split 30
Fitting LOOCV split 40
Fitting LOOCV split 50

Accuracy: 0.865

Variable Importance Measurements:
       Feature  Importance
35    F1-small    0.002515
66     du-call    0.002071
11     F1-call    0.001701
1      F1-also    0.001183
17     F1-frog    0.001109
15      F1-for    0.001036
30      F1-red    0.001036
63      du-bob    0.000888
83  du-plastic    0.000888
44    F1-these    0.000888


In [7]:
all_data = h_input(df)
all_data.process('country')
all_data.select_features(selected_features=['F1', 'F2', 'F3', 'duration'])
all_data.normalize(method='z')
all_data.select_places(places=['uk', 'usa'])

,id,gender,age,location,F1-a,F1-also,F1-and,F1-ask,F1-at,F1-bags,...,du-thick,du-things,du-three,du-to,du-toy,du-train,du-we,du-wednesday,du-will,du-with
0,english1,1,42.0,usa,-0.093493,0.083396,-0.114430,-0.431054,-0.512552,-0.602915,...,-1.680075,0.174453,-0.542345,-0.111229,-0.622436,0.190065,-0.537000,0.713914,-0.151216,-0.180224
1,english10,0,35.0,usa,-0.558218,1.256401,-0.347753,-0.286063,0.039436,0.842772,...,-0.451220,0.476209,-0.897073,-0.111229,-0.421244,0.190065,-0.071309,0.713914,-0.151216,-0.180224
2,english102,1,22.0,usa,-0.457330,-0.308033,-1.225896,-0.577729,-0.854514,-0.812310,...,-0.451220,-0.278182,-0.542345,-0.300555,0.383521,-0.156492,0.394382,0.713914,-0.151216,-0.896641
3,english103,1,21.0,usa,-0.629858,-0.512364,-0.657671,-1.788803,-1.157086,-1.597866,...,-1.270456,-1.183450,-0.897073,-0.111229,-1.829583,-1.889275,-0.769845,-0.578604,-0.151216,-0.538432
4,english104,0,18.0,usa,0.822831,1.101806,0.758018,1.500919,-0.442092,2.045183,...,-0.041602,-0.730816,-0.010254,0.078097,-1.024818,0.536621,-0.304154,0.067655,-0.151216,0.536194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,english95,1,18.0,usa,-1.460076,-0.643747,-1.142385,-1.469438,-0.418330,-0.819030,...,-0.041602,0.627087,-0.364981,1.024727,-0.220053,-0.156492,0.394382,0.067655,-0.151216,-0.180224
479,english96,1,31.0,usa,-1.350795,-1.004267,-0.258835,-1.170892,-0.816509,-1.068868,...,-0.451220,-0.127303,-0.719709,-0.111229,-1.226010,0.190065,-1.701227,-0.578604,-0.151216,-0.538432
480,english97,1,42.0,usa,-0.972930,0.054839,1.443173,-0.868070,-0.899393,-0.547513,...,-1.680075,-0.579938,-1.251800,-0.300555,-0.823627,-0.849605,-0.537000,0.067655,-0.151216,-0.896641
481,english98,1,22.0,usa,-0.909073,-0.148067,-0.594197,-0.751833,-1.649009,-0.708644,...,-0.860838,-0.881694,-0.719709,-0.300555,-0.622436,-0.503049,-0.537000,-1.871122,-0.151216,-0.180224


In [8]:
print("Real location:", all_data.input_df.iloc[305].location)
m.sample_predict(custom=all_data.input_df.iloc[305])

Real location: uk
Predicted location: uk


In [9]:
df[df.word=="snow"]

,id,gender,age,location,midpoint_time,phoneme,word,duration,F1,F2,F3,F1_onglide,F2_onglide,F1_offglide,F2_onglide.1
59,english1,m,42.0,pittsburgh_pennsylvania_usa,5.645,S,snow,0.11,1396.831372,1482.167083,2796.221370,933.8340693427355,2216.685297677341,1363.942066,1903.339769
60,english1,m,42.0,pittsburgh_pennsylvania_usa,5.725,N,snow,0.05,369.343279,1577.665511,2572.580652,893.9259993777534,2234.7081851151697,436.023012,1621.625116
61,english1,m,42.0,pittsburgh_pennsylvania_usa,5.800,OW1,snow,0.10,551.186692,1444.215991,2611.103677,571.8733467156159,1542.8584676680205,502.735754,1300.542475
278,english10,f,35.0,davenport_iowa_usa,6.250,S,snow,0.08,1641.538415,2931.255327,4240.554698,713.6067330876716,2469.1727653561884,854.776799,2525.061170
279,english10,f,35.0,davenport_iowa_usa,6.325,N,snow,0.07,339.830337,1201.066884,2722.731865,252.21513695563817,2199.986335349866,349.400579,1725.694841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140172,english98,m,22.0,spokane_washington_usa,6.360,N,snow,0.06,264.190537,1315.975411,2545.733113,228.80789057526457,1375.0806682726245,269.607783,1256.665485
140173,english98,m,22.0,spokane_washington_usa,6.450,OW1,snow,0.12,459.449305,1249.492524,2289.183135,431.7066491461994,1412.8889043332917,406.601040,1176.815908
140390,english99,f,52.0,pittsburgh_pennsylvania_usa,5.450,S,snow,0.08,2540.575694,3003.125323,3961.690104,2445.8191690699005,2907.1134737177877,2332.560077,3026.430816
140391,english99,f,52.0,pittsburgh_pennsylvania_usa,5.520,N,snow,0.06,393.280392,1938.622204,3874.839015,313.6819136217437,2064.6994635520323,386.794534,1730.640835
